__AASB Alinhamento global, local e Múltiplo__

Algoritmos de alinhamento global e local entre sequências e reconstrução do respetivo alinhamento   
Alinhamento Múltiplo

In [4]:
from threading import local

In [5]:
seq1 = "ATCTCT"
seq2 = "ATCG"

In [36]:
def criar_matriz(seq1,seq2):
    """
    Inputs:
    seq1 = sequência 1 a alinhar
    seq2 = sequência 2 a alinhar
    
    Output: matriz numérica de 0
    """
    mat=[]
    for i in range(len(seq1)):
        mat.append([])
        for j in range(len(seq2)):
            mat[i].append(0)
    return mat

In [7]:
mat = criar_matriz(seq1,seq2)
mat

[[0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0]]

In [35]:
def criar_matriz_seq(seq1,seq2): 
    """
    Input:
    seq1 = sequência 1 a alinhar
    seq2 = sequência 2 a alinhar
    
    Output: matriz numérica de 0 e 1
    """
    mat=criar_matriz(seq1,seq2)
    for i in range(len(seq1)):
        for j in range(len(seq2)):
            if seq1[i]==seq2[j]:
                mat[i][j]=1
            else:
                mat[i][j]=0
    return mat

In [7]:
mat = criar_matriz_seq(seq1,seq2)
mat

[[1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0],
 [0, 0, 1, 0],
 [0, 1, 0, 0]]

In [8]:
def imprimir_matriz(mat,seq1,seq2):
    """
    Input:
    mat = matriz numérica criada a partir função criar_matriz_seq
    seq1 = sequência 1 a alinhar
    seq2 = sequência 2 a alinhar
    
    Output: matriz numérica onde um * representam os caracteres iguais (match) de ambas as sequências
    """
    print(" "+seq2)
    for i in range (len(mat)):
        lin=[]
        lin.append(seq1[i])
        for j in range (len(mat[i])):
            if mat[i][j]>= 1:
                lin.append("*")
            else:
                lin.append(" ")
        print("".join(lin))

In [9]:
imprimir_matriz(mat,seq1,seq2)

 ATCG
A*   
T *  
C  * 
T *  
C  * 
T *  


In [37]:
def score(seq1, seq2): 
    """
    Inputs:
    seq1 = sequência 1 a alinhar
    seq2 = sequência 2 a alinhar
    
    Output: score
    """
    if seq1 == seq2: 
        return 3 
    else: return -1 

In [38]:
def alin_local(seq1, seq2, g):
    """
    Inputs:
    seq1 = sequência 1 a alinhar
    seq2 = sequência 2 a alinhar
    g = valor do gap
    
    Output: matriz de alinhamento local Smith-Waterman; matriz com direções 
    """
    mat = [[0 for C in range(len(seq2) + 1)] for L in range(len(seq1) + 1)]
    drt = [[" " for C in range(len(seq2) + 1)] for L in range(len(seq1) + 1)]
    for L in range(len(seq1)):
        mat[L + 1][0] =mat[L][0] + g
        drt[L + 1][0] = "C"
    for C in range(len(seq2)):
        mat[0][C + 1] =mat[0][C] + g
        drt[0][C + 1] = "E"
    if local:
        for L in range(len(seq1)):
            mat[L + 1][0] = 0
            drt[L + 1][0] = "0"
        for C in range(len(seq2)):
            mat[0][C + 1] = 0
            drt[0][C + 1] = "0"
    for L, x1 in enumerate(seq1):
        for C, x2 in enumerate(seq2):
            valor=[mat[L][C] + score(x1, x2), 
                    mat[L + 1][C    ] + g, 
                    mat[L    ][C + 1] + g] 

            if local: valor.append(0)
            direcoes = "D E C 0".split()
            mat[L + 1][C + 1] = max(valor)
            drt[L + 1][C + 1] = direcoes[valor.index(mat[L + 1][C + 1])]
            if sorted(valor)[-1] == sorted(valor)[-2]:
                if valor[0] == valor[1]: drt[L + 1][C + 1] = "DE"
                if valor[0] == valor[2]: drt[L + 1][C + 1] = "DC"
                if valor[1] == valor[2]: drt[L + 1][C + 1] = "EC"
                if valor[0] == valor[1] and valor[0] == valor[2]: drt[L + 1][C + 1] = "DCE"

    return mat, drt

In [27]:
alin_local(seq1, seq2, g=-1)

([[0, 0, 0, 0, 0],
  [0, 3, 2, 1, 0],
  [0, 2, 6, 5, 4],
  [0, 1, 5, 9, 8],
  [0, 0, 4, 8, 8],
  [0, 0, 3, 7, 7],
  [0, 0, 3, 6, 6]],
 [[' ', '0', '0', '0', '0'],
  ['0', 'D', 'E', 'E', 'DC'],
  ['0', 'C', 'D', 'E', 'E'],
  ['0', 'C', 'C', 'D', 'E'],
  ['0', 'DE', 'DC', 'C', 'D'],
  ['0', '0', 'C', 'DC', 'DC'],
  ['0', '0', 'D', 'C', 'DC']])

In [39]:
def alin_global(seq1, seq2, g):
    """
    Inputs:
    seq1 = sequência 1 a alinhar
    seq2 = sequência 2 a alinhar
    g = valor do gap
    
    Output: matriz de alinhamento global Needleman-Wunsch; matriz com direções 
    """
    mat = [[0 for C in range(len(seq2) + 1)] for L in range(len(seq1) + 1)]
    drt = [[" " for C in range(len(seq2) + 1)] for L in range(len(seq1) + 1)]
    for L in range(len(seq1)):
        mat[L + 1][0] =mat[L][0] + g
        drt[L + 1][0] = "C"
    for C in range(len(seq2)):
        mat[0][C + 1] =mat[0][C] + g
        drt[0][C + 1] = "E"
    for L, x1 in enumerate(seq1):
        for C, x2 in enumerate(seq2):
            valor=[mat[L][C] + score(x1, x2), 
                    mat[L + 1][C    ] + g, 
                    mat[L    ][C + 1] + g] 

            direcoes = "D E C 0".split()
            mat[L + 1][C + 1] = max(valor)
            drt[L + 1][C + 1] = direcoes[valor.index(mat[L + 1][C + 1])]
            if sorted(valor)[-1] == sorted(valor)[-2]:
                if valor[0] == valor[1]: drt[L + 1][C + 1] = "DE"
                if valor[0] == valor[2]: drt[L + 1][C + 1] = "DC"
                if valor[1] == valor[2]: drt[L + 1][C + 1] = "EC"
                if valor[0] == valor[1] and valor[0] == valor[2]: drt[L + 1][C + 1] = "DCE"

    return mat, drt

In [32]:
alin_global(seq1, seq2, g=-1)

([[0, -1, -2, -3, -4],
  [-1, 3, 2, 1, 0],
  [-2, 2, 6, 5, 4],
  [-3, 1, 5, 9, 8],
  [-4, 0, 4, 8, 8],
  [-5, -1, 3, 7, 7],
  [-6, -2, 2, 6, 6]],
 [[' ', 'E', 'E', 'E', 'E'],
  ['C', 'D', 'E', 'E', 'E'],
  ['C', 'C', 'D', 'E', 'E'],
  ['C', 'C', 'C', 'D', 'E'],
  ['C', 'C', 'DC', 'C', 'D'],
  ['C', 'C', 'C', 'DC', 'DC'],
  ['C', 'C', 'DC', 'C', 'DC']])

In [127]:
seqs = ["ATGG", "ATTT", "ATG"]

In [149]:
def alinhamento_progressivo(seqs: list, g):
    """
    Inputs:
    seqs = lista de sequências a alinhar
    g = valor do gap
    
    Output: matriz de alinhamento múltiplo 
    """
    def NW(seq1, seq2):
        alinhamento = alin_global(seq1, seq2, g=-1)
        S1 = alinhamento[0]
        S2 = alinhamento[1]
        return S1, S2
    
    def consenso(*seqs):
        return "".join(x1 if x1 == x2 else x1 if x2 == "-" else x2 for x1, x2 in zip(seq1,seq2))

    def alin_mult(seqs):
        seq1, seq2, *sequencias = seqs
        S1, S2 = NW(seq1, seq2)
        res = [S1, S2]
        cons = consenso(S1, S2)
        for p in sequencias:
            S1, S2 = NW(cons, p)
            res.append(S2)
            cons = consenso(S1, S2)

        res = []
        for j in seqs:
            S1, S2 = NW(cons, j)
            res.append(S2)
        return res
    return alin_mult(seqs)

In [150]:
alinhamento_progressivo(seqs, g=-1)

[[[' ', 'E', 'E', 'E', 'E'],
  ['C', 'D', 'E', 'E', 'E'],
  ['C', 'C', 'D', 'E', 'E'],
  ['C', 'C', 'C', 'D', 'DE'],
  ['C', 'C', 'C', 'D', 'D']],
 [[' ', 'E', 'E', 'E', 'E'],
  ['C', 'D', 'E', 'E', 'E'],
  ['C', 'C', 'D', 'DE', 'DE'],
  ['C', 'C', 'C', 'D', 'DE'],
  ['C', 'C', 'C', 'DC', 'D']],
 [[' ', 'E', 'E', 'E'],
  ['C', 'D', 'E', 'E'],
  ['C', 'C', 'D', 'E'],
  ['C', 'C', 'C', 'D'],
  ['C', 'C', 'C', 'D']]]